# Question Answering Model 
## no trainer

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [3]:
from datasets import load_dataset, disable_caching
from transformers import (
    AutoTokenizer,
    default_data_collator,
    get_scheduler,
    AutoModelForQuestionAnswering
)
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch
from torch.cuda import amp

from collections import defaultdict, Counter
from tqdm.auto import tqdm
import numpy as np

# for badge_query
from scipy import stats
from sklearn.metrics import pairwise_distances
import pdb

import os
import re
import string

arguments.py

In [4]:
# STRATEGY_NAME = 'RandomSampling'
STRATEGY_NAME = 'LeastConfidence'
# STRATEGY_NAME = 'MarginSampling'
# STRATEGY_NAME = 'BALDDropout'
DATA_NAME = 'DROP'
# EXPE_ROUND = 5
MODEL_BATCH = 8
MAX_LENGTH = None
LEARNING_RATE = 3e-5
MODEL_NAME = 'RoBERTa'
LOW_RES = True
NUM_TRAIN_EPOCH = 3
UNIQ_CONTEXT = True
if LOW_RES:
    args_input_quota = 100
    NUM_QUERY = 50
else:
    NUM_INIT_LB = 500 # 1000
    args_input_quota = 2000 # 200
    NUM_QUERY = 500 # 50
# ITERATION = int(args_input_quota / NUM_QUERY)

stride = 128

MODEL_DIR = '/mount/studenten/studentenarbeiten/linku/AL_QA/dev_models'
CACHE_DIR = '/mount/studenten/studentenarbeiten/linku/AL_QA/.cache'

preprocess.py

In [5]:
def preprocess_training_examples(examples, tokenizer):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


def preprocess_training_features(examples, tokenizer):
    # keep ["offset_mapping"] and ["example_id"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []
    contexts = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        # added for used in unlabel data predict
        example_ids.append(examples["id"][sample_idx]) 
        # added for unique context filter
        contexts.append(examples["context"][sample_idx])

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["context"] = contexts
    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

def preprocess_training_examples_lowRes(examples, tokenizer):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    detected_answers = examples["detected_answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = detected_answers[sample_idx]
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_training_features_lowRes(examples, tokenizer):
    # keep ["offset_mapping"] and ["example_id"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["detected_answers"]
    start_positions = []
    end_positions = []
    example_ids = []
    contexts = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)
        
        # added for used in unlabel data predict
        example_ids.append(examples["qid"][sample_idx])
        # added for unique context filter
        contexts.append(examples['context'][sample_idx])

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["context"] = contexts
    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples_lowRes(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["qid"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs


utils.py

In [6]:
def get_aubc(quota, bsize, resseq):
	# it is equal to use np.trapz for calculation
	ressum = 0.0
	if quota % bsize == 0:
		for i in range(len(resseq)-1):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2

	else:
		for i in range(len(resseq)-2):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2
		k = quota % bsize
		ressum = ressum + ((resseq[-1] + resseq[-2]) * k / 2)
	ressum = round(ressum / quota,3)
	
	return ressum


def get_mean_stddev(datax):
	return round(np.mean(datax),4),round(np.std(datax),4)


def get_unlabel_data(n_pool, labeled_idxs, train_dataset):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_idxs, unlabeled_data


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)


def get_model(m):
	if m.lower() == 'bert':
		return 'bert-base-uncased'
	elif m.lower() == 'bertlarge':
		return 'bert-large-uncased'
	elif m.lower() == 'roberta':
		return 'roberta-base'
	elif m.lower() == 'robertalarge':
		return 'roberta-large'

def preprocess_data(train_data, val_data):
	tokenizer = AutoTokenizer.from_pretrained(get_model(MODEL_NAME))

	if LOW_RES:
		train_dataset = train_data.map(
			preprocess_training_examples_lowRes,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		train_features = train_data.map(
			preprocess_training_features_lowRes,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_dataset = val_data.map(
			preprocess_validation_examples_lowRes,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_features = val_data.map(
			preprocess_validation_examples_lowRes,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
	else:
		train_dataset = train_data.map(
			preprocess_training_examples,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		train_features = train_data.map(
			preprocess_training_features,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_dataset = val_data.map(
			preprocess_validation_examples,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_features = val_data.map(
			preprocess_validation_examples,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)

	train_dataset.set_format("torch")
	train_features.set_format("torch")
	val_dataset = val_dataset.remove_columns(["offset_mapping"])
	val_dataset.set_format("torch")
	val_features.set_format("torch")

	return train_dataset, train_features, val_dataset, val_features


def load_dataset_mrqa(d):
	'''
	return train_set, val_set
	'''
	data = load_dataset("mrqa", cache_dir=CACHE_DIR)
	if d == 'squad':
		# the first to 86588th in train set
		# the first to 10507th in val set
		squad_train = data['train'].select(range(86588))
		squad_val = data['validation'].select(range(10507))
		for t in squad_train: assert t['subset'] == 'SQuAD', 'Please select corrrect train data for SQuAD.'
		for v in squad_val: assert v['subset'] == 'SQuAD', 'Please select corrrect validation data for SQuAD.'
		return squad_train, squad_val
	elif d == 'newsqa':
		# the 86589th to 160748th in train set
		# the 10508th to 14719th in val set
		data_set = data['train'].select(range(86588, 160748))
		newsqa_train = data_set.shuffle(1127).select(range(10000))
		newsqa_val = data['validation'].select(range(10507, 14719))
		for t in newsqa_train: assert t['subset'] == 'NewsQA', 'Please select corrrect train data for NewQA.'
		for v in newsqa_val: assert v['subset'] == 'NewsQA', 'Please select corrrect validation data for NewQA.'
		return newsqa_train, newsqa_val
	elif d == 'searchqa':
		# the 222437th to 339820th in train set
		# the 22505th to 39484th in val set
		data_set = data['train'].select(range(222436, 339820))
		searchqa_train = data_set.shuffle(1127).select(range(10000))
		searchqa_val = data['validation'].select(range(22504, 39484))	
		for t in searchqa_train: assert t['subset'] == 'SearchQA', 'Please select corrrect train data for SearchQA.'
		for v in searchqa_val: assert v['subset'] == 'SearchQA', 'Please select corrrect validation data for SearchQA.'
		return searchqa_train, searchqa_val
	elif d == 'bioasq':
		# the first to the 1504th in the test set
		sub = data['test'].select(range(1504))
		len_sub_val = len(sub) // 10
		bioasq_train = sub.select(range(len_sub_val, len(sub)))
		bioasq_val = sub.select(range(len_sub_val))
		for t in bioasq_train: assert t['subset'] == 'BioASQ', 'Please select corrrect train data for BioASQ.'
		for v in bioasq_val: assert v['subset'] == 'BioASQ', 'Please select corrrect validation data for BioASQ.'
		return bioasq_train, bioasq_val
	elif d == 'textbookqa':
		# the 8131st to 9633rd
		sub = data['test'].select(range(8130, 9633))
		len_sub_val = len(sub) // 10
		textbookqa_train = sub.select(range(len_sub_val, len(sub)))
		textbookqa_val = sub.select(range(len_sub_val)) 
		for t in textbookqa_train: assert t['subset'] == 'TextbookQA', 'Please select corrrect train data for TextbookQA.'
		for v in textbookqa_val: assert v['subset'] == 'TextbookQA', 'Please select corrrect validation data for TextbookQA.'
		return textbookqa_train, textbookqa_val
	elif d == 'drop': # Discrete Reasoning Over Paragraphs
		# the 1505th to 3007th in test set
		sub = data['test'].select(range(1504, 3007))
		len_sub_val = len(sub) // 10
		drop_train = sub.select(range(len_sub_val, len(sub)))
		drop_val = sub.select(range(len_sub_val))
		for t in drop_train: assert t['subset'] == 'DROP', 'Please select corrrect train data for DROP.'
		for v in drop_val: assert v['subset'] == 'DROP', 'Please select corrrect validation data for DROP.'
		return drop_train, drop_val
	

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluation(theoretical_answers, predicted_answers, skip_no_answer=False):
    '''
	theoretical_answers, datatype=dict
	{strings of id: list of ground truth answers}
	predicted_answers, datatype=dict
	{strings of id: strings of prediction text}
	'''
    f1 = exact_match = total = 0
    for qid, ground_truths in theoretical_answers.items():
        if qid not in predicted_answers:
            if not skip_no_answer:
                message = 'Unanswered question %s will receive score 0.' % qid
                print(message)
                total += 1
            continue
        total += 1
        prediction = predicted_answers[qid]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}


def save_model(device, pretrain_dir, strategy_dir):
    '''
    Copy and save model from pretrain_models to current trained models.
    '''
    pretrain_model = AutoModelForQuestionAnswering.from_pretrained(pretrain_dir).to(device)
    model_to_save = pretrain_model.module if hasattr(pretrain_model, 'module') else pretrain_model 
    model_to_save.save_pretrained(strategy_dir)


def init_centers(X, K):
    ind = np.argmax([np.linalg.norm(s, 2) for s in X])
    mu = [X[ind]]
    indsAll = [ind]
    centInds = [0.] * len(X)
    cent = 0
    print('#Samps\tTotal Distance')
    while len(mu) < K:
        if len(mu) == 1:
            D2 = pairwise_distances(X, mu).ravel().astype(float)
        else:
            newD = pairwise_distances(X, [mu[-1]]).ravel().astype(float)
            for i in range(len(X)):
                if D2[i] >  newD[i]:
                    centInds[i] = cent
                    D2[i] = newD[i]
        print(str(len(mu)) + '\t' + str(sum(D2)), flush=True)
        if sum(D2) == 0.0: pdb.set_trace()
        D2 = D2.ravel().astype(float)
        Ddist = (D2 ** 2)/ sum(D2 ** 2)
        customDist = stats.rv_discrete(name='custm', values=(np.arange(len(D2)), Ddist))
        ind = customDist.rvs(size=1)[0]
        while ind in indsAll: ind = customDist.rvs(size=1)[0]
        mu.append(X[ind])
        indsAll.append(ind)
        cent += 1
    return indsAll

def query(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i):
	if STRATEGY_NAME == 'RandomSampling':
		iter_i_labeled_idxs = random_sampling(n_pool, labeled_idxs, train_features, i)
	# elif STRATEGY_NAME == 'MarginSampling':
	# 	iter_i_labeled_idxs = margin(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	elif STRATEGY_NAME == 'LeastConfidence':
		iter_i_labeled_idxs = least_confidence(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'EntropySampling':
	# 	iter_i_labeled_idxs = entropy(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'MarginSamplingDropout':
	# 	iter_i_labeled_idxs = margin_dropout(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'LeastConfidenceDropout':
	# 	iter_i_labeled_idxs = least_confidence_dropout(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'EntropySamplingDropout':
	# 	iter_i_labeled_idxs = entropy_dropout(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'BALDDropout':
	# 	iter_i_labeled_idxs = bald(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'BatchBALDDropout':
	# 	iter_i_labeled_idxs = batch_bald(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'MeanSTD':
	# 	iter_i_labeled_idxs = mean_std(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	# elif STRATEGY_NAME == 'KMeansSampling':
	# 	iter_i_labeled_idxs = kmeans(n_pool, labeled_idxs, train_dataset, train_features, device, i)
	# elif STRATEGY_NAME == 'KCenterGreedy':
	# 	if args_input.low_resource and i == 1:
	# 		iter_i_labeled_idxs = random_sampling(n_pool, labeled_idxs, train_features, i)
	# 	else:
	# 		iter_i_labeled_idxs = kcenter(n_pool, labeled_idxs, train_dataset, train_features, device, i)
	# elif STRATEGY_NAME == 'BadgeSampling':
	# 	iter_i_labeled_idxs = badge(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
	else:
		raise NotImplementedError
	return iter_i_labeled_idxs

In [7]:
# move get_us_uc() for developing

def get_us(labeled_idxs, score_ordered_idxs, n_pool, features, iteration=0):
	if LOW_RES:
		total = NUM_QUERY * iteration
		print('Total num of label pool in LowRes:', total)
	else:
		total = NUM_QUERY * iteration + NUM_INIT_LB
		print('Total num of label pool in regular:', total)  

	# count if we have enough unique sample to select
	labeled_idxs_ = labeled_idxs.copy()
	labeled_idxs_[score_ordered_idxs] = True
	samples_ = features.select(indices=np.arange(n_pool)[labeled_idxs_])
	ssi_ = set(samples_['example_id']) 
	print('\nWe have {} unique ssi in scored pool.\n'.format(len(ssi_)))

	# create select_sample_id(ssi) set
	labeled_idxs[score_ordered_idxs[:NUM_QUERY]] = True
	samples = features.select(indices=np.arange(n_pool)[labeled_idxs])
	ssi = set(samples['example_id']) 
	print('We have {} unique ssi.'.format(len(ssi)))

	sliced_till = NUM_QUERY
	while len(ssi) < total:
		difference = total - len(ssi)
		print('Not enough ssi, still need {} ssi.'.format(difference))
		labeled_idxs[score_ordered_idxs[sliced_till:sliced_till + difference]] = True	# get extra
		sliced_till += difference
		samples = features.select(indices=np.arange(n_pool)[labeled_idxs])
		for sample in samples:
			ssi.add(sample['example_id'])
		print('End of add extra ssi, now we have {} unique ssi.'.format(len(ssi)))
    
	labeled_idxs[score_ordered_idxs[:sliced_till]] = True
	return np.arange(n_pool)[labeled_idxs]

### load dataset

In [8]:
disable_caching()
if LOW_RES:
	init_pool = 0
	setting = 'low resource'
	## set dir
	pretrain_model_dir = os.path.abspath('') + '/pretrain_models' + '/' + MODEL_NAME + '_SQuAD_full_dataset_lr_3e-5'
	strategy_model_dir = MODEL_DIR + '/lowRes_' + str(args_input_quota) + '_' + STRATEGY_NAME + '_' + MODEL_NAME +  '_' + DATA_NAME
	## load data
	train_data, val_data = load_dataset_mrqa(DATA_NAME.lower())
else:
	init_pool = NUM_INIT_LB
	setting = 'regular'
	## set dir
	strategy_model_dir = MODEL_DIR + '/' + str(NUM_INIT_LB) + '_' + str(args_input_quota) + '_' + STRATEGY_NAME + '_' + MODEL_NAME +  '_' + DATA_NAME
	## load data
	squad = load_dataset(DATA_NAME.lower(), cache_dir=CACHE_DIR)
	
	train_data = squad["train"]
	val_data = squad["validation"]
	# print('Use full training data and full testing data.')

Found cached dataset mrqa (/mount/studenten/studentenarbeiten/linku/AL_QA/.cache/mrqa/plain_text/1.1.0/1f2cf5ac32b43f864e6f91d384057a16b69b7d13ba9bcaa200ac277c90938d19)


  0%|          | 0/3 [00:00<?, ?it/s]

model.py

In [9]:
def to_train(num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler, record_loss=False):
	if LOW_RES:
		print('Training was performed using {} query data, i.e. {} data.'.format(NUM_QUERY, len(train_dataloader.dataset)))
	else:
		print('Training was performed using the sum of {} initial data and {} query data, i.e. {} data.'.format(NUM_INIT_LB, NUM_QUERY, len(train_dataloader.dataset)))
	
	for epoch in range(num_train_epochs):
		model.train()
		for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
			batch = {key: value.to(device) for key, value in batch.items()}
			outputs = model(**batch)
			loss = outputs.loss
			loss.backward()

			optimizer.step()
			lr_scheduler.step()
			optimizer.zero_grad()

		if record_loss:
			print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch, loss.item()))

	model_to_save = model.module if hasattr(model, 'module') else model 
	model_to_save.save_pretrained(strategy_model_dir)
	print('TRAIN done!')

def to_pretrain(num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler, scaler):
	print('Training was performed using the full dataset ({} data).'.format(len(train_dataloader.dataset)))
	for epoch in range(num_train_epochs):
		model.train()
		for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
			batch = {key: value.to(device) for key, value in batch.items()}
			with amp.autocast():
				outputs = model(**batch)
				loss = outputs.loss
			
			scaler.scale(loss).backward()

			scaler.step(optimizer)
			scaler.update()
			lr_scheduler.step()
			optimizer.zero_grad()

		print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch, loss.item()))

	model_to_save = model.module if hasattr(model, 'module') else model 
	model_to_save.save_pretrained(pretrain_model_dir)
	print('TRAIN done!')

import evaluate
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

def compute_metrics_lowRes(start_logits, end_logits, features, examples):
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = dict()
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["qid"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers[example_id] = best_answer["text"]
        else:
            predicted_answers[example_id] = ""

    theoretical_answers = dict()
    for ex in examples: theoretical_answers[ex["qid"]] = ex["answers"]
    return evaluation(theoretical_answers, predicted_answers)

def get_pred(dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.eval()
    start_logits = []
    end_logits = []

    for batch in tqdm(dataloader, desc="Evaluating_pred"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    if LOW_RES:
        return compute_metrics_lowRes(start_logits, end_logits, features, examples)
    else:
        return compute_metrics(start_logits, end_logits, features, examples)

def get_pretrain_pred(dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(pretrain_model_dir).to(device)
    
    model.eval()
    start_logits = []
    end_logits = []

    for batch in tqdm(dataloader, desc="Evaluating_pred"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    return compute_metrics(start_logits, end_logits, features, examples)

def get_prob(dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)

    model.eval()
    start_logits = []
    end_logits = []

    for batch in tqdm(dataloader, desc="Evaluating_prob"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    prob_dict = {}
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20
    
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    for example in tqdm(examples):
        if LOW_RES:
            example_id = example["qid"]
        else:
            example_id = example["id"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(start_logit[start_index] + end_logit[end_index])
        
            if len(answers) > 1:
                prob_dict[feature_index] = softmax(answers)
            elif example_to_features[example_id] != []:
                prob_dict[feature_index] = np.array([0])
    
    return prob_dict

def get_prob_dropout(dataloader, device, features, examples, n_drop=10):
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.train()
    prob_dict = {}
    
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        for batch in tqdm(dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples):
            if LOW_RES:
                example_id = example["qid"]
            else:
                example_id = example["id"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            if 1 < len(answers) < 200: # pad to same numbers of possible answers
                zero_list = [0] * (200 - len(answers))
                answers.extend(zero_list)
            elif len(answers) >= 200:
                answers = answers[:200]

            if len(answers) > 1:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = softmax(answers)
                else:
                    prob_dict[example_to_features[example_id][0]] += softmax(answers)
            elif example_to_features[example_id] != []:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = np.array([0])   

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop

    return prob_dict

def get_prob_dropout_split(dataloader, device, features, examples, n_drop=10):
    ## use tensor to save the answers

    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.train()

    probs = torch.zeros([n_drop, len(dataloader.dataset), 200])
    
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        for batch in tqdm(dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples, desc="Computing metrics"):
            if LOW_RES:
                example_id = example["qid"]
            else:
                example_id = example["id"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            
                if 1 < len(answers) < 200: # pad to same numbers of possible answers
                    zero_list = [0] * (200 - len(answers))
                    answers.extend(zero_list)
                elif len(answers) >= 200:
                    answers = answers[:200]

                probs[i][feature_index] += torch.tensor(softmax(answers))

    return probs

def get_embeddings(dataloader, device):
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir, output_hidden_states=True).to(device)
    
    model.eval()
    embeddings = torch.zeros([len(dataloader.dataset), model.config.to_dict()['hidden_size']])
    idxs_start = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating_prob"):
            batch = {key: value.to(device) for key, value in batch.items()}
        
            outputs = model(**batch)

            hidden_states = outputs.hidden_states
            embedding_of_last_layer = hidden_states[-2][:, 0, :]

            idxs_end = idxs_start + len(hidden_states[-2])
            embeddings[idxs_start:idxs_end] = embedding_of_last_layer.cpu()
            idxs_start = idxs_end
        
    return embeddings

def get_grad_embeddings(dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir, output_hidden_states=True).to(device)
    
    model.eval()

    nLab = 20
    embDim = model.config.to_dict()['hidden_size']
    embeddings = np.zeros([len(dataloader.dataset), embDim * nLab])

    prob_dict = []
    idxs_start = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating_prob"):
            batch = {key: Variable(value.to(device)) for key, value in batch.items()}
                
            # deepAL+: out, e1 = self.clf(x)
            outputs = model(**batch)
            # deepAL+: e1 = e1.data.cpu().numpy()
            hidden_states = outputs.hidden_states
            embedding_of_last_layer = hidden_states[-2][:, 0, :]
            embedding_of_last_layer = embedding_of_last_layer.data.cpu().numpy()

            # matually create features batch
            data_len_batch = len(outputs.start_logits)
            idxs_end = idxs_start + data_len_batch
            batch_idx = list(range(idxs_start, idxs_end))
            batch_feat = features.select(batch_idx)
            idxs_start = idxs_end

            out = logits_to_prob(outputs.start_logits.cpu().numpy(), outputs.end_logits.cpu().numpy(), batch_feat, batch_idx, examples, 200)
            batchProbs = F.softmax(out, dim=1).data.cpu().numpy()
            maxInds = np.argmax(batchProbs, 1)

            for j in range(data_len_batch):
                for c in range(nLab):
                    if c == maxInds[j]:
                        embeddings[batch_idx[j]][embDim * c : embDim * (c+1)] = deepcopy(embedding_of_last_layer[j]) * (1 - batchProbs[j][c]) * -1.0
                    else:
                        embeddings[batch_idx[j]][embDim * c : embDim * (c+1)] = deepcopy(embedding_of_last_layer[j]) * (-1 * batchProbs[j][c]) * -1.0
            
    return embeddings

def logits_to_prob(start_logits, end_logits, features, batch_idx, examples, num_classes):
    probs = torch.zeros([len(batch_idx), num_classes])
    
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append((idx, batch_idx[idx]))
    
    for example in tqdm(examples, desc="Computing metrics"):
        if LOW_RES:
            example_id = example["qid"]
        else:
            example_id = example["id"]
        answers = []
        
        # Loop through all features associated with that example
        for (feature_index, i) in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    answers.append(start_logit[start_index] + end_logit[end_index])


            if 1 < len(answers) < num_classes: # pad to same numbers of possible answers
                zero_list = [0] * (num_classes - len(answers))
                answers.extend(zero_list)
            elif len(answers) >= num_classes:
                answers = answers[:num_classes]
            probs[feature_index] = torch.tensor(answers)

    return probs

query.py

In [10]:
def random_sampling(n_pool, labeled_idxs, features, i):
    print('Random querying starts.')

    score_ordered_idxs = np.random.choice(np.where(labeled_idxs==0)[0], NUM_QUERY*2, replace=False)
    if UNIQ_CONTEXT:
        iter_i_labeled_idxs = get_us_uc(labeled_idxs, score_ordered_idxs, n_pool, features, i)
    else:
        iter_i_labeled_idxs = get_us(labeled_idxs, score_ordered_idxs, n_pool, features, i)

    return iter_i_labeled_idxs

def least_confidence(n_pool, labeled_idxs, dataset, features, examples, device, i):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, dataset)
    unlabeled_features = features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		collate_fn=default_data_collator,
		batch_size=MODEL_BATCH,
	)

    print('LC querying starts.')
    print('Query {} data from {} unlabeled training data.\n'.format(NUM_QUERY, len(unlabeled_data)))

    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    print('Got probability.')

    confidence_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            confidence_dict[idx] = max(probs)
        elif idx:
            confidence_dict[idx] = np.array([0])

    sorted_confidence_list = sorted(confidence_dict.items(), key=lambda x: x[1])
    score_ordered_idxs = unlabeled_idxs[[idx for (idx, _) in sorted_confidence_list[:NUM_QUERY*2]]]
    
    if UNIQ_CONTEXT:
        iter_i_labeled_idxs = get_us_uc(labeled_idxs, score_ordered_idxs, n_pool, features, i)
    else:
        iter_i_labeled_idxs = get_us(labeled_idxs, score_ordered_idxs, n_pool, features, i)

    return iter_i_labeled_idxs


Next we will preprocess the dataset (training and evaluation data).

In [11]:
train_dataset, train_features, val_dataset, val_features = preprocess_data(train_data, val_data)

Map:   0%|          | 0/1353 [00:00<?, ? examples/s]

Map:   0%|          | 0/1353 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Set seed and device

In [12]:
SEED = 1127
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

all_acc = []
all_acc_dict = {}
all_acc_em_dict = {}
acq_time = []

In [13]:
print(len(train_dataset))
print(len(train_features))

1415
1415


Experiment Round 1

In [14]:
EXP_ROUND = 1

In [15]:
## record acc performance 
# acc = np.zeros(ITERATION + 1) # quota/batch runs + iter_0
# acc_em = np.zeros(ITERATION + 1)
acc = np.zeros(4) # quota/batch runs + iter_0
acc_em = np.zeros(4)

## generate initial labeled pool
n_pool = len(train_dataset)
labeled_idxs = np.zeros(n_pool, dtype=bool)


if LOW_RES:
    save_model(device, pretrain_model_dir, strategy_model_dir)
else:
    tmp_idxs = np.arange(n_pool)
    np.random.shuffle(tmp_idxs)

    
    if UNIQ_CONTEXT:
        iter_0_labeled_idxs = get_us_uc(labeled_idxs, tmp_idxs, n_pool, train_features)
    else:
        iter_0_labeled_idxs = get_us(labeled_idxs, tmp_idxs, n_pool, train_features)

    ## load the selected train data to DataLoader
    train_dataloader = DataLoader(
        train_dataset.select(indices=iter_0_labeled_idxs),
        shuffle=True,
        collate_fn=default_data_collator,
        batch_size=MODEL_BATCH,
    )

    num_update_steps_per_epoch = len(train_dataloader)
    num_training_steps = NUM_TRAIN_EPOCH * num_update_steps_per_epoch

    ## network
    model = AutoModelForQuestionAnswering.from_pretrained(get_model(MODEL_NAME)).to(device)
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    ## iteration 0 accuracy
    to_train(NUM_TRAIN_EPOCH, train_dataloader, device, model, optimizer, lr_scheduler)

## load the selected validation data to DataLoader
eval_dataloader = DataLoader(
    val_dataset, 
    collate_fn=default_data_collator, 
    batch_size=MODEL_BATCH
)

acc_scores_0 = get_pred(eval_dataloader, device, val_features, val_data) # add i=1 to use model from models_dir
acc[0] = round(acc_scores_0['f1'], 4)
acc_em[0] = round(acc_scores_0['exact_match'], 4)

print('\nIterantion 0 done.')
print('Testing accuracy: {}'.format(acc[0]))
print('Testing accuracy em {}'.format(acc_em[0]))
# time = datetime.datetime.now()
# print('Time spent for init training:', (time - start))

Evaluating_pred:   0%|          | 0/19 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/150 [00:00<?, ?it/s]


Iterantion 0 done.
Testing accuracy: 49.5043
Testing accuracy em 40.6667


In [16]:
i = 1
print('Iteraion {} in exp_round_{} start.'.format(i, EXP_ROUND))
		
## query
# iter_i_labeled_idxs = query(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
## iter_i_labeled_idxs = least_confidence(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, i)
unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
unlabeled_features = train_features.select(unlabeled_idxs)
unlabeled_dataloader = DataLoader(
    unlabeled_data,
    collate_fn=default_data_collator,
    batch_size=MODEL_BATCH,
)

print('LC querying starts.')
print('Query {} data from {} unlabeled training data.\n'.format(NUM_QUERY, len(unlabeled_data)))

prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, train_data)
print('Got probability.')

confidence_dict = {}
for idx, probs in prob_dict.items():
    if len(probs) > 1: # if prob_dict['probs'] is not 0
        confidence_dict[idx] = max(probs)
    elif idx:
        confidence_dict[idx] = np.array([0])

sorted_confidence_list = sorted(confidence_dict.items(), key=lambda x: x[1])
score_ordered_idxs = unlabeled_idxs[[idx for (idx, _) in sorted_confidence_list[:NUM_QUERY*2]]]

Iteraion 1 in exp_round_1 start.
LC querying starts.
Query 50 data from 1415 unlabeled training data.



Evaluating_prob:   0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/1353 [00:00<?, ?it/s]

Got probability.


In [17]:
def get_us_uc(labeled_idxs, score_ordered_idxs, n_pool, features, iteration=0):
	if LOW_RES:
		total = NUM_QUERY * iteration
		print('Total num of label pool in LowRes:', total)
	else:
		total = NUM_QUERY * iteration + NUM_INIT_LB
		print('Total num of label pool in regular:', total)

	ssi = set()
	uc = set()
	
	samples = features.select(indices=np.arange(n_pool)[labeled_idxs])
	if len(samples):
		for sample in samples:
			ssi.add(sample['example_id'])
			uc.add(sample['context'])
	assert len(ssi) == len(samples), "The amount of ssi from previous query is wrong. There are only {} ssi.".format(len(ssi))
	assert len(uc) == len(samples), "The amount of uc from previous query is wrong. There are only {} uc.".format(len(uc))
	
	filtered_score_ordered_idx = []
	for soi in score_ordered_idxs:
		pool_idxs = np.zeros(len(features), dtype=bool)
		pool_idxs[soi] = True
		sample = features.select(indices=np.arange(n_pool)[pool_idxs])

		if sample[0]['example_id'] not in ssi:
			if sample[0]['context'] not in uc:
				ssi.add(sample[0]['example_id'])
				uc.add(sample[0]['context'])
				filtered_score_ordered_idx.append(soi)

		if len(filtered_score_ordered_idx) == total:
			print('Break the loop, we have {} unique ssi having unique context.'.format(len(ssi)))
			break
			
	print('\nFinally, we have {} instances.'.format(len(filtered_score_ordered_idx)))
	labeled_idxs[filtered_score_ordered_idx] = True
	return np.arange(n_pool)[labeled_idxs]

In [18]:
if UNIQ_CONTEXT:
    iter_i_labeled_idxs = get_us_uc(labeled_idxs, score_ordered_idxs, n_pool, train_features, i)
else:
    iter_i_labeled_idxs = get_us(labeled_idxs, score_ordered_idxs, n_pool, train_features, i)

Total num of label pool in LowRes: 50
Break the loop, we have 50 unique ssi having unique context.

Finally, we have 50 instances.
